# Approach taken

Using a csv file with the data about the books inside project gutenburg, generate a summary about the book from prompt engineering - calling a gemini-pro model once every 5-6 seconds.

 Taking this summary and embedding the information using the gte-small model in chunks using the SpacyTextSplitter. This embedding is stored in the vector database implemented using langchain's SupabaseVectorStore to insert it into the database. Other metadata is also stored in a traditional relational database for future use.

 This data is used in the Supabase edge function to get the semantic search on books and book graph working.

In [ ]:
!pip install langchain
!pip install sentence_transformers
!pip install supabase
!pip install -U langchain-community
!pip install langchain_huggingface
!pip install tqdm
!pip install openai
!pip install --quiet langchain-google-genai
!pip install langchain
!pip install sentence_transformers
!pip install supabase
!pip install -U langchain-community
!pip install langchain_huggingface
!pip install tqdm
!pip install openai
!pip install --quiet langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.1 MB/s eta 0:00:00
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from supabase import create_client, Client
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import SupabaseVectorStore
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from time import sleep
from langchain_text_splitters.spacy import SpacyTextSplitter
from langchain.prompts import PromptTemplate
from time import sleep
from langchain_google_genai import ChatGoogleGenerativeAI
import spacy
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Run this cell and paste the API key in the prompt
import os
import getpass
os.environ['GOOGLE_API_KEY'] = getpass.getpass('Gemini API Key:')

Gemini API Key:··········


In [ ]:
# Set up environment variables
supabase_url = "https://rgjkrflnxopeixwpsjae.supabase.co"
supabase_key = os.environ['SUPABASE_KEY']
openai_api_key = ""  # Replace with your OpenAI API key

# Set the OpenAI API key
#os.environ["OPENAI_API_KEY"] = openai_api_key

# Initialize Supabase client
supabase: Client = create_client(supabase_url, supabase_key)

# Initialize embedding function
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

# Initialize text splitter
#text_splitter = SpacyTextSplitter(chunk_size=512, chunk_overlap=100)


text_splitter = SpacyTextSplitter(chunk_size=128, chunk_overlap=32, separator="\n", pipeline="en_core_web_sm")


# # Initialize LLM
# llm = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


# If there is no env variable set for API key, you can pass the API key
# to the parameter `google_api_key` of the `ChatGoogleGenerativeAI` function:
# `google_api_key="key"`.

llm = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85)

# Define prompt template
prompt_template = PromptTemplate(
    template="""
    Generate a concise 400 word summary for the book "{title}" by {author}. Focus on key plot points, themes, characters, and writing style. Ensure the summary contains important keywords to facilitate effective semantic search matching.
    """,
    input_variables=["title", "author"]
)



# Load the CSV file
csv_path = 'sample_data/all_books.csv'
csv_data = pd.read_csv(csv_path)

def generate_summary(prompt):
    print("generating summary for prompt", prompt)
    response = llm.invoke(prompt);
    print("llm response: ", response)
    summary = response.content.strip()
    return summary

def populate_tables():
    print("populate tables")
    subjects = {}
    bookshelves = {}
    locc_codes = {}

    # Fetch existing subjects, bookshelves, and locc codes from Supabase
    subjects = {item['name']: item['id'] for item in supabase.table('subjects').select('id', 'name').execute().data}
    bookshelves = {item['name']: item['id'] for item in supabase.table('bookshelves').select('id', 'name').execute().data}
    locc_codes = {item['code']: item['id'] for item in supabase.table('locc').select('id', 'code').execute().data}

    vector_store = SupabaseVectorStore(embedding=embedding_model,
                                client=supabase,
                                table_name="documents",
                                query_name="match_documents")


    for i, row in tqdm(csv_data.iloc[383:].iterrows(), total=csv_data.shape[0] - 383):
    #for i, row in tqdm(csv_data.iterrows(), total=csv_data.shape[0]):
        book_id = str(row['Text#'])
        title = str(row['Title'])
        author = str(row['Authors'])
        raw_subjects = str(row['Subjects'])
        raw_bookshelves = str(row['Bookshelves'])
        raw_locc = str(row['LoCC'])

        print("i = {}, Bookid = {}, Title = {}, Author = {}".format(i, book_id, title, author))

        prompt = prompt_template.format(title=title, author=author)
        # Generate summary
        summary = generate_summary(prompt)

        if summary == '':
            print("Summary is empty, skipping")
            continue



        # Insert book into books table
        book_data = {
            'book_id': book_id,
            'title': title,
            'author': author,
            'summary': summary,
            'metadata' : {
                'subjects': raw_subjects,
                'bookshelves': raw_bookshelves,
                'locc': raw_locc
            }
            #'embedding': embedding
        }
        docs = text_splitter.create_documents([summary], [book_data])
        try:
            book_id = supabase.table('books').upsert(book_data).execute().data[0]['book_id']
            print("inserted book into table")
        except Exception as e:
            print("Error inserting book into table:", e)
        try:
            vector_store.add_documents(docs)
            print("inserted document into vector store")
        except Exception as e:
            print("Error inserting document into vector store:", e)

        # Process subjects
        if not raw_subjects:
          for subject in raw_subjects.split('--'):
              if subject and subject not in subjects:
                  subject_data = {'name': subject}
                  subject_id = supabase.table('subjects').upsert(subject_data).execute().data[0]['id']
                  subjects[subject] = subject_id

              # Map book to subject in book_subjects table
              book_subject_data = {'book_id': book_id, 'subject_id': subjects[subject]}
              supabase.table('book_subjects').upsert(book_subject_data).execute()

        if not raw_bookshelves:
          # Process bookshelves
          for bookshelf in raw_bookshelves.split('--'):
              if bookshelf and bookshelf not in bookshelves:
                  bookshelf_data = {'name': bookshelf}
                  bookshelf_id = supabase.table('bookshelves').upsert(bookshelf_data).execute().data[0]['id']
                  bookshelves[bookshelf] = bookshelf_id

              # Map book to bookshelf in book_bookshelves table
              book_bookshelf_data = {'book_id': book_id, 'bookshelf_id': bookshelves[bookshelf]}
              supabase.table('book_bookshelves').upsert(book_bookshelf_data).execute()

        if not raw_locc:
          # Process locc
          for locc in raw_locc.split('; '):
              if locc and locc not in locc_codes:
                  locc_data = {'code': locc}
                  locc_id = supabase.table('locc').upsert(locc_data).execute().data[0]['id']
                  locc_codes[locc] = locc_id

              # Map book to locc in book_locc table
              book_locc_data = {'book_id': book_id, 'locc_id': locc_codes[locc]}
              supabase.table('book_locc').upsert(book_locc_data).execute()

        print("Book_id {} done, going to sleep for 5 sec".format(book_id))
        sleep(5)  # Add a delay to avoid hitting rate limits

# Populate tables
populate_tables()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


populate tables


  0%|          | 0/11027 [00:00<?, ?it/s]

i = 383, Bookid = 961, Title = Glinda of Oz
In Which Are Related the Exciting Experiences of Princess Ozma of Oz, and Dorothy, in Their Hazardous Journey to the Home of the Flatheads, and to the Magic Isle of the Skeezers, and How They Were Rescued from Dire Peril by the Sorcery of Glinda the Good, Author = Baum, L. Frank (Lyman Frank), 1856-1919
generating summary for prompt 
    Generate a concise 400 word summary for the book "Glinda of Oz
In Which Are Related the Exciting Experiences of Princess Ozma of Oz, and Dorothy, in Their Hazardous Journey to the Home of the Flatheads, and to the Magic Isle of the Skeezers, and How They Were Rescued from Dire Peril by the Sorcery of Glinda the Good" by Baum, L. Frank (Lyman Frank), 1856-1919. Focus on key plot points, themes, characters, and writing style. Ensure the summary contains important keywords to facilitate effective semantic search matching.
    


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


...

i = 1068, Bookid = 3407, Title = The Spell of Egypt, Author = Hichens, Robert, 1864-1950
generating summary for prompt 
    Generate a concise 400 word summary for the book "The Spell of Egypt" by Hichens, Robert, 1864-1950. Focus on key plot points, themes, characters, and writing style. Ensure the summary contains important keywords to facilitate effective semantic search matching.
    


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


llm response:  content='**Summary of "The Spell of Egypt" by Robert Hichens**\n\n"The Spell of Egypt" is a captivating novel that immerses readers in the alluring and enigmatic land of Egypt.\n\n**Plot:**\n\nThe story follows Horace Villiers, a young Englishman drawn to Egypt by its ancient mysteries. Horace encounters a cast of intriguing characters, including the alluring Lady Euphrosyne, who becomes his obsession. As Horace delves deeper into the country\'s spell, he faces supernatural forces and uncovers a hidden world of ancient rituals and secrets.\n\n**Themes:**\n\nThe novel explores themes of cultural clash, the allure of the exotic, and the power of ancient beliefs. Hichens deftly contrasts the modern world of Victorian England with the timeless and mystical Egypt, highlighting the profound impact of the country\'s history and culture on the human psyche.\n\n**Characters:**\n\n* **Horace Villiers:** A young Englishman seeking adventure and spiritual enlightenment in Egypt.\n* 

  6%|▌         | 685/11027 [2:20:27<35:20:40, 12.30s/it]


KeyboardInterrupt: 

#Second approach

Populate a bunch of metadata from the csv file with project gutenburg metadata

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from supabase import create_client, Client
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from time import sleep

# Set up environment variables
supabase_url = "https://rgjkrflnxopeixwpsjae.supabase.co"
supabase_key = os.environ['SUPABASE_KEY']

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = openai_api_key

# Initialize Supabase client
supabase: Client = create_client(supabase_url, supabase_key)

# Initialize embedding function
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

# Initialize LLM
llm = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Define prompt template
prompt_template = PromptTemplate(
    template="""
    Generate a concise 400 word summary for the book "{title}" by {author}. Focus on key plot points, themes, characters, and writing style. Ensure the summary contains important keywords to facilitate effective semantic search matching.
    """,
    input_variables=["title", "author"]
)

# Load the CSV file
csv_path = 'sample_data/all_books.csv'
csv_data = pd.read_csv(csv_path)

def generate_summaries(titles_and_authors):
    print("generating summaries")
    prompts = [prompt_template.format(title=title, author=author) for title, author in titles_and_authors]
    responses = llm.batch(prompts, max_tokens=1024, temperature=0.7)
    summaries = [response["choices"][0]["text"].strip() for response in responses]
    return summaries

def populate_tables():
    print("populate tables")
    subjects = {}
    bookshelves = {}
    locc_codes = {}
    book_data_list = []
    titles_and_authors = []

    for _, row in tqdm(csv_data.iterrows(), total=csv_data.shape[0]):
        book_id = str(row['Text#'])
        title = str(row['Title'])
        author = str(row['Authors'])
        raw_subjects = str(row['Subjects'])
        raw_bookshelves = str(row['Bookshelves'])
        raw_locc = str(row['LoCC'])

        titles_and_authors.append((title, author))
        print(len(titles_and_authors))

    # Generate summaries in batch
    summaries = generate_summaries(titles_and_authors)

    for idx, (_, row) in enumerate(tqdm(csv_data.iterrows(), total=csv_data.shape[0])):
        print("generating embeddings")

        book_id = str(row['Text#'])
        title = str(row['Title'])
        author = str(row['Authors'])
        raw_subjects = str(row['Subjects'])
        raw_bookshelves = str(row['Bookshelves'])
        raw_locc = str(row['LoCC'])

        # Get the summary from the pre-generated summaries
        summary = summaries[idx]

        # Generate embedding
        embedding = embedding_model.embed_query(summary)

        # Prepare book data for bulk insertion
        book_data = {
            'title': title,
            'author': author,
            'summary': summary,
            'embedding': embedding
        }
        book_data_list.append(book_data)

        # Insert subjects, bookshelves, and locc in their respective tables
        for subject in raw_subjects.split('--'):
            if subject and subject not in subjects:
                subject_data = {'name': subject}
                subject_id = supabase.table('subjects').upsert(subject_data).execute().data[0]['id']
                subjects[subject] = subject_id

            # Map book to subject in book_subjects table
            book_subject_data = {'book_id': book_id, 'subject_id': subjects[subject]}
            supabase.table('book_subjects').insert(book_subject_data).execute()

        for bookshelf in raw_bookshelves.split('--'):
            if bookshelf and bookshelf not in bookshelves:
                bookshelf_data = {'name': bookshelf}
                bookshelf_id = supabase.table('bookshelves').upsert(bookshelf_data).execute().data[0]['id']
                bookshelves[bookshelf] = bookshelf_id

            # Map book to bookshelf in book_bookshelves table
            book_bookshelf_data = {'book_id': book_id, 'bookshelf_id': bookshelves[bookshelf]}
            supabase.table('book_bookshelves').insert(book_bookshelf_data).execute()

        for locc in raw_locc.split('; '):
            if locc and locc not in locc_codes:
                locc_data = {'code': locc}
                locc_id = supabase.table('locc').upsert(locc_data).execute().data[0]['id']
                locc_codes[locc] = locc_id

            # Map book to locc in book_locc table
            book_locc_data = {'book_id': book_id, 'locc_id': locc_codes[locc]}
            supabase.table('book_locc').insert(book_locc_data).execute()

        sleep(20)  # Add a delay to avoid hitting rate limits

    # Bulk insert book data
    supabase.table('books').insert(book_data_list).execute()

# Populate tables
populate_tables()


# First Approach

Tried to process the entire book with no chunking

In [ ]:
import os
import requests
import pandas as pd
import uuid
from bs4 import BeautifulSoup
from supabase import create_client, Client
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
from langchain.vectorstores import SupabaseVectorStore
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

# Set up environment variables
supabase_url = "https://rgjkrflnxopeixwpsjae.supabase.co"
supabase_key = os.environ['SUPABASE_KEY']


# Initialize Supabase client
supabase: Client = create_client(supabase_url, supabase_key)

# Initialize embedding functionload_dotenv()
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Load the CSV file
csv_path = 'pg_catalog.csv'
csv_data = pd.read_csv(csv_path)


Code for getting data from project gutenburg - should be helpful to load other data for books

In [ ]:
def fetch_book_data(book_id):
    metadata_url = f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.rdf"
    response = requests.get(metadata_url)
    if response.status_code != 200:
        print(f"Failed to retrieve metadata for book {book_id}")
        return

    soup = BeautifulSoup(response.content, 'lxml')

    title_tag = soup.find('dcterms:title')
    author_tag = soup.find('pgterms:name')

    if not title_tag or not author_tag:
        print(f"Skipping book {book_id} due to missing metadata")
        return

    title = title_tag.text
    author = author_tag.text
    book_url = f"https://www.gutenberg.org/cache/epub/{book_id}"

    text_url = f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt.utf8"
    text_response = requests.get(text_url)
    if text_response.status_code != 200:
        print(f"Failed to retrieve text for book {book_id}")
        return

    text_content = text_response.text

    # Generate embeddings for the text content
    embeddings = embedding_function.encode(text_content).tolist()

    return {
        "content" : text_content[],
        "embeddings" : embeddings,
        "author" : author,
        "url" : book_url,
        "title" : title
        }

    #store_book(book_id, title, author, book_url, text_url, tags, embeddings)


In [ ]:
# import os
# import pandas as pd
# import uuid
# from supabase import create_client, Client
# from dotenv import load_dotenv
# from tqdm import tqdm
# from langchain.vectorstores import SupabaseVectorStore
# from langchain_core.documents import Document
# from langchain_huggingface import HuggingFaceEmbeddings


# Load environment variables
load_dotenv()

embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Load the CSV file
csv_path = 'pg_catalog.csv'
csv_data = pd.read_csv(csv_path)



# def fetch_book_data(book_id):
#     try:
#         book_response = supabase.table('books').select('*').eq('book_id', book_id).single().execute()
#         embedding_response = supabase.table('book_embeddings').select('*').eq('book_id', book_id).single().execute()

#         return book_response.data, embedding_response.data

#     except Exception as e:
#         print(f"Exception occurred while fetching data for book_id {book_id}: {e}")
#         return None, None


def create_metadata(book_id, book_data,):
    return {
        'book_id': book_id,
        'title': book_data.get('title', ''),
        'author': book_data.get('author', ''),
        'url': book_data.get('url', ''),
        'subjects' : subjects,
        'bookshelves' : bookshelves
    }

def populate_documents():
    docs = []
    for i, row in tqdm(csv_data.iterrows(), total=csv_data.shape[0]):
        book_id = str(row['Text#'])
        subjects = str(row['Subjects'])
        bookshelves = str(row['Bookshelves'])
        book_data = fetch_book_data(book_id)
        if book_data:
            metadata = create_metadata(book_id, book_data, subjects, bookshelves)
            embedding = book_data['embedding']
            content = book_data['content']

            document = Document(
                page_content=content,
                metadata=metadata,
                embedding=embeddingex
            )

        try:
            vector_store.add_documents([document])
            print(f"Inserted document for book_id {book_id}")
        except Exception as e:
            print(f"Error inserting document for book_id {book_id}: {e}")

if __name__ == "__main__":
    populate_documents()


In [ ]:
def store_book_metadata(book_id, title, author, url):
    supabase.table('books').insert({
        'book_id': book_id,
        'title': title,
        'author': author,
        'url': url
    }).execute()

def store_book_embeddings(book_id, embeddings):
    supabase.table('book_embeddings').insert({
        'book_id': book_id,
        'embeddings': embeddings
    }).execute()

def scrape_book_metadata_and_generate_embeddings(book_id):
    metadata_url = f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.rdf"
    response = requests.get(metadata_url)
    if response.status_code != 200:
        print(f"Failed to retrieve metadata for book {book_id}")
        return

    soup = BeautifulSoup(response.content, 'lxml')

    title_tag = soup.find('dcterms:title')
    author_tag = soup.find('pgterms:name')

    if not title_tag or not author_tag:
        print(f"Skipping book {book_id} due to missing metadata")
        return

    title = title_tag.text
    author = author_tag.text
    book_url = f"https://www.gutenberg.org/cache/epub/{book_id}"

    text_url = f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt.utf8"
    text_response = requests.get(text_url)
    if text_response.status_code != 200:
        print(f"Failed to retrieve text for book {book_id}")
        return

    text_content = text_response.text

    # Generate embeddings for the text content
    embeddings = embedding_function.encode(text_content).tolist()

    store_book_metadata(book_id, title, author, book_url)
    store_book_embeddings(book_id, embeddings)
    print(f"Stored metadata and embeddings for book {book_id}")


In [ ]:
# Example usage
for book_id in tqdm(range(673, 73756)):  # Adjust range as needed
    scrape_book_metadata_and_generate_embeddings(book_id)